In [1]:
using BenchmarkTools

In [2]:
include("../GameAnalysis.jl")

batch_nash (generic function with 1 method)

In [3]:
players = 30
actions = 4
functions = 200
# points_per_dim = 89
agg = additive_sin_game(players, actions, functions, sin_params=Dict(:period_range=>[0,100],:amplitude_range=>[0,1000]))
# pay_dict = to_sym_game(agg, PayoffDict)
pay_arr = to_sym_game(agg, PayoffArrays)
# dev_prof = to_sym_game(agg, DeviationProfiles)

PayoffArrays(30, 4, [30 0 0 0; 29 1 0 0; … ; 0 0 1 29; 0 0 0 30], [354.60123870905875 505.3499222817151 505.3499222817151 505.3499222817151; 352.9294832692765 340.8327555180106 505.3499222817151 505.3499222817151; … ; 505.3499222817151 505.3499222817151 657.347942572738 919.3995489943939; 505.3499222817151 505.3499222817151 505.3499222817151 894.7679111095053], 9658.752532563016, 0.05232041307384206)

In [4]:
points_per_dim = 17
grid = mixture_grid(4,points_per_dim)


4×969 Matrix{Float64}:
 1.0  0.9375  0.9375  0.9375  0.875  …  0.0   0.0     0.0    0.0     0.0
 0.0  0.0625  0.0     0.0     0.125     0.0   0.0     0.0    0.0     0.0
 0.0  0.0     0.0625  0.0     0.0       0.25  0.1875  0.125  0.0625  0.0
 0.0  0.0     0.0     0.0625  0.0       0.75  0.8125  0.875  0.9375  1.0

In [11]:
@belapsed deviation_payoffs($pay_arr, $grid) evals=1 samples=1

8.929171562

In [9]:
@belapsed deviation_payoffs($pay_arr, $grid) evals=1 samples=4

8.983069682

In [10]:
@belapsed deviation_payoffs($pay_arr, $grid) evals=4 samples=1

8.96466431625

In [ ]:
function batched_dev_pays(game, mixtures, batch_size)
    num_mixtures = size(mixtures,2)
    dev_pays = Array{Float64}(undef, size(mixtures)...)
    for start_index in 1:batch_size:num_mixtures
        end_index = min(start_index + batch_size - 1, num_mixtures)
        dev_pays[:,start_index:end_index] = deviation_payoffs(game, mixtures[:,start_index:end_index])
    end
    return dev_pays
end

In [ ]:
@benchmark batched_dev_pays($pay_dict, $grid, 1)

In [ ]:
@benchmark batched_dev_pays($pay_arr, $grid, 1)

In [ ]:
@benchmark batched_dev_pays($dev_prof, $grid, 1)

In [ ]:
println("PayoffDict: ", game_size(PayoffDict, players, actions))
println("PayoffArrays: ", game_size(PayoffArrays, players, actions))
println("DeviationProfiles: ", game_size(DeviationProfiles, players, actions))

In [ ]:
584896.0000000014 / 1.1697920000000028e6


In [ ]:
584896.0000000014 / 1.697920000000028e6